In [1]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl')
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

from dl.nets.layers import TimeDistributed


In [2]:
mount_point = '/mnt/raid/C1_ML_Analysis'

importlib.reload(us_simu)
vs = us_simu.VolumeSamplingBlindSweep(mount_point=mount_point)
vs.init_probe_params()
vs = vs.cuda()

In [3]:

# diffusor = sitk.ReadImage('/mnt/famli_netapp_shared/C1_ML_Analysis/src/blender/simulated_data_export/studies_merged/FAM-025-0447-5.nrrd')
# diffusor_np = sitk.GetArrayFromImage(diffusor)
# diffusor_t = torch.tensor(diffusor_np.astype(int))

# diffusor_spacing = torch.tensor(diffusor.GetSpacing()).flip(dims=[0])
# diffusor_size = torch.tensor(diffusor.GetSize()).flip(dims=[0])

# diffusor_origin = torch.tensor(diffusor.GetOrigin()).flip(dims=[0])
# diffusor_end = diffusor_origin + diffusor_spacing * diffusor_size
# print(diffusor_size)
# print(diffusor_spacing)
# print(diffusor_t.shape)
# print(diffusor_origin)
# print(diffusor_end)

# diffusor_np, diffusor_head = nrrd.read('/mnt/raid//C1_ML_Analysis/simulated_data_export/placenta/FAM-025-0664-4_label11_resampled.nrrd')
diffusor_np, diffusor_head = nrrd.read(os.path.join('/mnt/raid/C1_ML_Analysis/simulated_data_export/animation_export/', 'breech/frame_0001.nrrd'))
diffusor_t = torch.tensor(diffusor_np.astype(int)).permute(2, 1, 0)
print(diffusor_head)
diffusor_size = torch.tensor(diffusor_head['sizes'])
diffusor_spacing = torch.tensor(np.diag(diffusor_head['space directions']))

diffusor_origin = torch.tensor(diffusor_head['space origin']).flip(dims=[0])
diffusor_end = diffusor_origin + diffusor_spacing * diffusor_size
print(diffusor_spacing)
print(diffusor_t.shape)
print(diffusor_origin)
print(diffusor_end)


In [4]:
fig = px.imshow(diffusor_t.flip(dims=[1]).squeeze().cpu().numpy(), animation_frame=0, binary_string=True)
fig.show()

In [5]:
diffusor_batch_t = diffusor_t.permute([2, 1, 0]).cuda().float().unsqueeze(0).unsqueeze(0)


diffusor_origin_batch = diffusor_origin[None, :]
diffusor_end_batch = diffusor_end[None, :]

print(diffusor_batch_t.shape, diffusor_origin_batch.shape, diffusor_origin_batch.shape)
# print(diffusor_origin_batch.shape)

diffusor_in_fov_t = vs.diffusor_in_fov(diffusor_batch_t, diffusor_origin_batch.cuda(), diffusor_end_batch.cuda())


In [6]:
# fig = px.imshow(diffusor_in_fov_t[0].squeeze().flip(dims=[1]).cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()
# fig = px.imshow(diffusor_in_fov_t[1].squeeze().flip(dims=[1]).cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()

In [7]:
importlib.reload(us_simulation_jit)
us_simulator_cut = us_simulation_jit.MergedLinearLabel11()
us_simulator_cut = us_simulator_cut.cuda()
grid, inverse_grid, mask_fan = us_simulator_cut.init_grids(256, 256, 128.0, -30.0, 20.0, 215.0, 0.7853981633974483)
us_simulator_cut_td = TimeDistributed(us_simulator_cut, time_dim=2).eval().cuda()


In [8]:


t_idx = 5

# vs.tags = ['M', 'L0', 'L1', 'R0', 'R1', 'C1', 'C2', 'C3', 'C4']
tag = vs.tags[t_idx]


# sampled_sweep = vs.diffusor_sampling_tag(tag, diffusor_batch_t.to(torch.float).cuda(), diffusor_origin_batch.to(torch.float).cuda(), diffusor_end_batch.to(torch.float).cuda())
# sampled_sweep_simu = torch.cat([us_simulator_cut_td(ss.unsqueeze(dim=0), grid.cuda(), inverse_grid.cuda(), mask_fan.cuda()) for ss in sampled_sweep], dim=0)


# sampled_sweep_simu = vs.get_sweep(diffusor_batch_t.to(torch.float).cuda(), diffusor_origin_batch.to(torch.float).cuda(), diffusor_end_batch.to(torch.float).cuda(), tag, use_random=False, simulator=us_simulator_cut_td, grid=grid.cuda(), inverse_grid=inverse_grid.cuda(), 
sampled_sweep_simu = vs.get_sweep(diffusor_batch_t.to(torch.float).cuda(), diffusor_origin_batch.to(torch.float).cuda(), diffusor_end_batch.to(torch.float).cuda(), tag, use_random=False, simulator=None, grid=grid.cuda(), inverse_grid=inverse_grid.cuda(), mask_fan=mask_fan.cuda())
print(sampled_sweep_simu.shape)


In [12]:
fig = px.imshow(sampled_sweep_simu.squeeze().detach().cpu().numpy(), animation_frame=0, binary_string=True)
fig.show()

In [10]:
# mesh_grid_params = [torch.arange(start=-1.0, end=1.0, step=(2.0/s), device=sampled_sweep_simu.device) for s in sampled_sweep_simu.shape[-3:]]
# z, y, x = torch.meshgrid(mesh_grid_params, indexing='ij')
# mesh_grid = torch.stack([x, y, z], dim=-1).to(torch.float32).unsqueeze(0)

# repeats = [1,]*len(mesh_grid.shape)
# repeats[0] = sampled_sweep_simu.shape[0]

# mesh_grid = mesh_grid.repeat(repeats)

# sampled_sweep_simu_v2 = F.grid_sample(sampled_sweep_simu[:,0,:,85:120,:,:], mesh_grid, align_corners=True)

# fig = px.imshow(sampled_sweep_simu_v2.squeeze().detach().cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()

In [11]:

out_fovs = vs.simulated_sweep_in_fov(tag, sampled_sweep_simu[:,:,0,80:120,:,:]).detach()
# print(simulation_ultrasound_plane_mesh_grid_transformed_t.shape)

# fig = px.imshow(sampled_sweep[0].squeeze().cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()
# fig = px.imshow(sampled_sweep_simu[0].squeeze().cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()
# fig = px.imshow(out_fovs[0].flip(dims=[1]).squeeze().detach().cpu().numpy(), animation_frame=2, binary_string=True)
# fig.show()

In [ ]:
out_fovs = vs.simulated_sweep_in_fov(tag, sampled_sweep_simu[:,:,0,:,:,:]).detach()
# fig = px.imshow(out_fovs[0].flip(dims=[1]).squeeze().cpu().numpy(), animation_frame=2, binary_string=True)
# fig = px.imshow(out_fov.flip(dims=[0]).squeeze().cpu().numpy(), animation_frame=2, binary_string=True)
# fig.show()

In [ ]:
# diffusor_plane_t = diffusor_plane_t.squeeze().unsqueeze(1)

# diffusor_plane_t = vs.diffusor_sampling_tag('C1', diffusor_t.unsqueeze(0).unsqueeze(0).cuda().to(torch.float), diffusor_origin.cuda().to(torch.float), diffusor_end.cuda().to(torch.float))

# # print(diffusor_plane_t.shape)
# repeats = [1,]*4
# repeats[0] = diffusor_plane_t.shape[0]

# with torch.no_grad():
#     x = us_simulator_cut_td(diffusor_plane_t, grid.repeat(repeats).cuda(), inverse_grid.repeat(repeats).cuda(), mask_fan.repeat(repeats).cuda())

# fig = px.imshow(x.squeeze().cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()

# torch.cuda.empty_cache()


In [ ]:
sampled_sweep_simu_emb = vs.embed_sweep(tag, sampled_sweep_simu[:,:,0,80:120,:,:])
print(sampled_sweep_simu_emb.shape)
# sampled_sweep_simu_emb_v = sampled_sweep_simu_emb[:,:,99:100,:,:]
sampled_sweep_simu_emb_v = sampled_sweep_simu_emb
print(sampled_sweep_simu_emb_v.shape)
sampled_sweep_simu_emb_v = sampled_sweep_simu_emb_v.flatten(start_dim=2).permute(0, 2, 1)
print(sampled_sweep_simu_emb_v.shape)

# fig = px.imshow(sampled_sweep_simu_emb[0,0,100,:,:].cpu().numpy(), binary_string=True)
# fig.show()

In [ ]:
r_idx = torch.randint(0, sampled_sweep_simu_emb_v.shape[1], (65536,))
r_idx = torch.sort(r_idx)[0]

SN = 0
x_v = sampled_sweep_simu_emb_v[:, r_idx, :].detach().cpu().numpy()

fig = go.Figure(data=[go.Scatter3d(x=x_v[SN,:,1], y=x_v[SN,:,2], z=x_v[SN,:,3], mode='markers', marker=dict(
        size=2,
        color=x_v[SN,:,0],
        colorscale='jet',
        # opacity=0.5
    ))])
fig.show()

In [ ]:
vs.probe_origins_M.shape

In [ ]:
vs.probe_directions_M.shape

In [ ]:
vs.probe_directions_M[0]